# Phase 1: T⁶ Flat Torus - Pipeline Calibration

**Objectif**: Valider que le pipeline spectral reproduit λ₁ connu analytiquement

**Runtime**: CPU suffisant (GPU optionnel)

---

## Contexte Mathématique

Le 6-tore T⁶ = (S¹)⁶ a un spectre du Laplacien analytiquement connu:

$$\lambda_{n_1,...,n_6} = \sum_{i=1}^{6} \frac{n_i^2}{R_i^2}$$

Pour $R_i = 1$ sur chaque cercle:
- $\lambda_0 = 0$ (mode constant)
- $\lambda_1 = 1$ (premier mode excité: un seul $n_i = 1$)

**ATTENTION**: T⁶ a holonomie **triviale**, pas SU(3). 
Ce n'est PAS un test de la conjecture, juste une calibration du pipeline.

---

In [ ]:
# Cell 1: Setup
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
from scipy.spatial.distance import cdist
import json
from datetime import datetime
import time
import matplotlib.pyplot as plt

# GPU optionnel
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("✓ CuPy available - GPU acceleration enabled")
except ImportError:
    GPU_AVAILABLE = False
    print("✗ CuPy not available - using CPU (sufficient for calibration)")

print(f"NumPy version: {np.__version__}")

In [ ]:
# Cell 2: Constants & Config

# T⁶ analytique
LAMBDA1_EXACT = 1.0  # Pour R₁ = ... = R₆ = 1
DIM = 6

# T⁶ "comme CY₃" (nombres de Hodge limites)
H11, H21 = 9, 9  # Limite plate d'un CY₃ avec χ=0
CHI = 0

# Définitions H* (pour référence, pas utilisées dans calibration)
HSTAR_A = H11 + H21 + 2  # = 20
HSTAR_B = H11 + 2*H21 + 4  # = 31
HSTAR_C = abs(H11 - H21) + 2  # = 2

# Paramètres numériques
N_VALUES = [500, 1000, 2000, 5000, 10000]
K_VALUES = [15, 25, 40]
SEED = 42

# Critères PASS/FAIL
TOLERANCE = 0.20  # 20% de déviation max pour calibration

print(f"T⁶ Configuration:")
print(f"  Dimension: {DIM}")
print(f"  λ₁ exact: {LAMBDA1_EXACT}")
print(f"  H* (A/B/C): {HSTAR_A} / {HSTAR_B} / {HSTAR_C}")
print(f"  Tolérance: ±{TOLERANCE*100:.0f}%")

In [ ]:
# Cell 3: T⁶ Sampling Functions

def sample_T6_uniform(N: int, radii: list = None, seed: int = 42) -> np.ndarray:
    """
    Échantillonne N points uniformément sur T⁶ = [0, 2π)⁶
    
    Args:
        N: nombre de points
        radii: [R₁,...,R₆] rayons (défaut: tous égaux à 1)
        seed: graine aléatoire
    
    Returns:
        points: (N, 6) array de coordonnées angulaires
    """
    rng = np.random.default_rng(seed)
    
    if radii is None:
        radii = [1.0] * 6
    
    # Coordonnées angulaires uniformes [0, 2π)
    angles = rng.uniform(0, 2 * np.pi, size=(N, 6))
    
    return angles, np.array(radii)


def geodesic_distance_T6(angles: np.ndarray, radii: np.ndarray) -> np.ndarray:
    """
    Distance géodésique sur T⁶ (distance torique).
    
    La distance sur chaque S¹ est min(|θ₁-θ₂|, 2π - |θ₁-θ₂|).
    Distance totale = sqrt(sum of squared circle distances).
    """
    N = angles.shape[0]
    
    # Calculer toutes les différences angulaires
    # Shape: (N, N, 6)
    diff = np.abs(angles[:, None, :] - angles[None, :, :])
    
    # Distance torique sur chaque cercle: min(d, 2π - d)
    diff_toric = np.minimum(diff, 2 * np.pi - diff)
    
    # Pondérer par les rayons
    diff_toric_scaled = diff_toric * radii[None, None, :]
    
    # Distance euclidienne dans l'espace produit
    D = np.sqrt(np.sum(diff_toric_scaled**2, axis=2))
    
    return D.astype(np.float32)


def lambda1_T6_exact(radii: list = None) -> float:
    """
    λ₁ exact sur T⁶ = min_{n≠0} Σᵢ nᵢ²/Rᵢ²
    
    Le mode le plus bas non-constant a un seul nᵢ = 1, les autres = 0.
    Donc λ₁ = 1/R²_max
    """
    if radii is None:
        radii = [1.0] * 6
    
    return 1.0 / max(radii)**2


print("✓ T⁶ sampling functions defined")

In [ ]:
# Cell 4: Graph Laplacian

def build_graph_laplacian(D: np.ndarray, k: int = 25, 
                          laplacian_type: str = "symmetric") -> sp.csr_matrix:
    """
    Construit le Laplacien de graphe à partir de la matrice de distances.
    
    Args:
        D: (N, N) matrice de distances
        k: nombre de voisins
        laplacian_type: "symmetric", "random_walk", ou "unnormalized"
    
    Returns:
        L: Laplacien sparse
    """
    N = D.shape[0]
    k = min(k, N - 1)
    
    # Sigma = median des distances k-NN
    knn_dists = np.partition(D, k, axis=1)[:, 1:k+1]  # Exclure self (dist=0)
    sigma = max(np.median(knn_dists), 1e-10)
    
    # Poids gaussiens
    W = np.exp(-D**2 / (2 * sigma**2))
    np.fill_diagonal(W, 0)
    
    # Garder seulement k plus proches voisins
    for i in range(N):
        idx_keep = np.argpartition(-W[i], k)[:k]  # Top k
        mask = np.ones(N, dtype=bool)
        mask[idx_keep] = False
        W[i, mask] = 0
    
    # Symétriser
    W = (W + W.T) / 2
    
    # Degrés
    d = np.maximum(W.sum(axis=1), 1e-10)
    
    # Construire le Laplacien
    if laplacian_type == "unnormalized":
        L = np.diag(d) - W
    elif laplacian_type == "random_walk":
        d_inv = 1.0 / d
        L = np.eye(N) - np.diag(d_inv) @ W
    elif laplacian_type == "symmetric":
        d_inv_sqrt = 1.0 / np.sqrt(d)
        L = np.eye(N) - np.outer(d_inv_sqrt, d_inv_sqrt) * W
    else:
        raise ValueError(f"Unknown laplacian_type: {laplacian_type}")
    
    return sp.csr_matrix(L)


def compute_lambda1(L: sp.csr_matrix, n_eigenvalues: int = 5) -> float:
    """
    Calcule λ₁ (premier eigenvalue non-nul) du Laplacien.
    """
    try:
        eigs, _ = eigsh(L, k=n_eigenvalues, which='SM', tol=1e-8)
        eigs = np.sort(np.real(eigs))
        
        # Premier eigenvalue > seuil
        for ev in eigs:
            if ev > 1e-8:
                return float(ev)
        
        return float(eigs[1]) if len(eigs) > 1 else 0.0
        
    except Exception as e:
        print(f"Eigensolve error: {e}")
        return np.nan


print("✓ Graph Laplacian functions defined")

In [ ]:
# Cell 5: Quick Test
print("=" * 60)
print("  QUICK TEST: N=1000")
print("=" * 60)

t0 = time.time()

# Sample
angles, radii = sample_T6_uniform(1000, seed=SEED)
print(f"Sampled {len(angles)} points on T⁶")

# Distance matrix
D = geodesic_distance_T6(angles, radii)
print(f"Distance matrix: {D.shape}, range [{D.min():.3f}, {D.max():.3f}]")

# Laplacian & eigenvalue
L = build_graph_laplacian(D, k=25)
lambda1 = compute_lambda1(L)

# Compare to exact
lambda1_exact = lambda1_T6_exact(radii.tolist())
deviation = abs(lambda1 - lambda1_exact) / lambda1_exact * 100

print(f"\nResults:")
print(f"  λ₁ (measured): {lambda1:.4f}")
print(f"  λ₁ (exact):    {lambda1_exact:.4f}")
print(f"  Deviation:     {deviation:.2f}%")
print(f"  Time:          {time.time() - t0:.2f}s")

status = "PASS" if deviation < TOLERANCE * 100 else "FAIL"
print(f"\n  Status: [{status}]")

In [ ]:
# Cell 6: Convergence Study
print("=" * 60)
print("  CONVERGENCE STUDY: N vs λ₁")
print("=" * 60)

results = []
lambda1_exact = LAMBDA1_EXACT

for N in N_VALUES:
    print(f"\nN = {N}...")
    
    for k in K_VALUES:
        t0 = time.time()
        
        # Sample & compute
        angles, radii = sample_T6_uniform(N, seed=SEED)
        D = geodesic_distance_T6(angles, radii)
        L = build_graph_laplacian(D, k=k)
        lambda1 = compute_lambda1(L)
        
        elapsed = time.time() - t0
        deviation = abs(lambda1 - lambda1_exact) / lambda1_exact * 100
        
        result = {
            "N": N,
            "k": k,
            "lambda1": lambda1,
            "lambda1_exact": lambda1_exact,
            "deviation_pct": deviation,
            "passed": deviation < TOLERANCE * 100,
            "time_s": elapsed,
        }
        results.append(result)
        
        status = "✓" if result["passed"] else "✗"
        print(f"  k={k}: λ₁={lambda1:.4f} (dev={deviation:.1f}%) [{status}] [{elapsed:.1f}s]")

print("\n" + "=" * 60)

In [ ]:
# Cell 7: Visualization
print("Generating convergence plot...")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: λ₁ vs N for each k
ax1 = axes[0]
for k in K_VALUES:
    subset = [r for r in results if r["k"] == k]
    Ns = [r["N"] for r in subset]
    l1s = [r["lambda1"] for r in subset]
    ax1.plot(Ns, l1s, 'o-', label=f'k={k}')

ax1.axhline(y=LAMBDA1_EXACT, color='r', linestyle='--', label=f'λ₁ exact = {LAMBDA1_EXACT}')
ax1.axhspan(LAMBDA1_EXACT * (1 - TOLERANCE), LAMBDA1_EXACT * (1 + TOLERANCE), 
            alpha=0.2, color='green', label=f'±{TOLERANCE*100:.0f}% tolerance')
ax1.set_xlabel('N (number of points)')
ax1.set_ylabel('λ₁ (measured)')
ax1.set_title('T⁶ Calibration: λ₁ Convergence')
ax1.set_xscale('log')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Deviation vs N
ax2 = axes[1]
for k in K_VALUES:
    subset = [r for r in results if r["k"] == k]
    Ns = [r["N"] for r in subset]
    devs = [r["deviation_pct"] for r in subset]
    ax2.plot(Ns, devs, 'o-', label=f'k={k}')

ax2.axhline(y=TOLERANCE * 100, color='r', linestyle='--', label=f'PASS threshold ({TOLERANCE*100:.0f}%)')
ax2.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
ax2.set_xlabel('N (number of points)')
ax2.set_ylabel('Deviation from exact (%)')
ax2.set_title('T⁶ Calibration: Convergence Error')
ax2.set_xscale('log')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('T6_calibration_convergence.png', dpi=150)
plt.show()

print("✓ Saved: T6_calibration_convergence.png")

In [ ]:
# Cell 8: Summary & PASS/FAIL Decision
print("=" * 60)
print("  PHASE 1 SUMMARY: T⁶ CALIBRATION")
print("=" * 60)

# Count passes
n_pass = sum(1 for r in results if r["passed"])
n_total = len(results)
pass_rate = n_pass / n_total * 100

# Best result
best = min(results, key=lambda r: r["deviation_pct"])
worst = max(results, key=lambda r: r["deviation_pct"])

print(f"\nResults:")
print(f"  Tests run:      {n_total}")
print(f"  Tests passed:   {n_pass} ({pass_rate:.1f}%)")
print(f"  Best deviation: {best['deviation_pct']:.2f}% (N={best['N']}, k={best['k']})")
print(f"  Worst deviation: {worst['deviation_pct']:.2f}% (N={worst['N']}, k={worst['k']})")

# High-N results (most reliable)
high_N = [r for r in results if r["N"] >= 5000]
if high_N:
    high_N_dev = np.mean([r["deviation_pct"] for r in high_N])
    print(f"  Mean deviation (N≥5000): {high_N_dev:.2f}%")

# PASS/FAIL decision
print("\n" + "-" * 40)
calibration_passed = pass_rate >= 80  # Au moins 80% des tests passent

if calibration_passed:
    print("  PHASE 1 STATUS: [PASS]")
    print("  Pipeline calibré correctement sur T⁶.")
    print("  Prêt pour Phase 2: T⁶/ℤ₃ orbifold")
else:
    print("  PHASE 1 STATUS: [FAIL]")
    print("  ATTENTION: Le pipeline ne converge pas vers λ₁ exact.")
    print("  Vérifier: distance torique, normalisation Laplacien")

print("-" * 40)

In [ ]:
# Cell 9: Save Results
output = {
    "metadata": {
        "phase": "Phase 1: T⁶ Calibration",
        "timestamp": datetime.now().isoformat(),
        "manifold": "T6_flat",
        "lambda1_exact": LAMBDA1_EXACT,
        "tolerance": TOLERANCE,
        "N_values": N_VALUES,
        "k_values": K_VALUES,
        "seed": SEED,
    },
    "results": results,
    "summary": {
        "n_tests": n_total,
        "n_passed": n_pass,
        "pass_rate_pct": pass_rate,
        "best_deviation_pct": best["deviation_pct"],
        "worst_deviation_pct": worst["deviation_pct"],
        "calibration_passed": calibration_passed,
    },
    "next_phase": "Phase 2: T⁶/ℤ₃ orbifold" if calibration_passed else "Debug pipeline",
}

filename = "T6_calibration_results.json"
with open(filename, "w") as f:
    json.dump(output, f, indent=2)

print(f"\n✓ Saved: {filename}")
print("\nDownload this file and share with Claude!")

In [ ]:
# Cell 10: Preview H* definitions (for Phase 2+)
print("\n" + "=" * 60)
print("  PREVIEW: H* DEFINITIONS FOR FUTURE PHASES")
print("=" * 60)

# T⁶ values (h¹¹=9, h²¹=9)
print(f"\nT⁶ (flat limit): h¹¹=9, h²¹=9, χ=0")
print(f"  H*_A = {HSTAR_A} (h¹¹ + h²¹ + 2)")
print(f"  H*_B = {HSTAR_B} (h¹¹ + 2h²¹ + 4)")
print(f"  H*_C = {HSTAR_C} (|h¹¹ - h²¹| + 2)")

# Predictions if conjecture held (but T⁶ has trivial holonomy!)
print(f"\nSi conjecture λ₁×H* = 6 s'appliquait (MAIS holonomie triviale!):")
print(f"  λ₁ target (A) = 6/{HSTAR_A} = {6/HSTAR_A:.4f}")
print(f"  λ₁ target (B) = 6/{HSTAR_B} = {6/HSTAR_B:.4f}")
print(f"  λ₁ target (C) = 6/{HSTAR_C} = {6/HSTAR_C:.4f}")

print(f"\n  λ₁ mesuré = {best['lambda1']:.4f} (T⁶ flat)")
print(f"  Note: T⁶ a λ₁ = 1.0, PAS lié à H*")
print(f"\n  => La conjecture ne s'applique qu'aux holonomies spéciales (SU(3), G₂, etc.)")